<a href="https://colab.research.google.com/github/datarobot-community/mlops-examples/blob/master/MLOps%20DRUM/Main_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using MLOps DRUM to test your custom models
**Author**: Tim Whittaker

#### Scope
We'll get our hands dirty by:

* Building a simple regression model using Scikit
* Using DRUM for Batch Scoring
* Using DRUM to get a REST API endpoint
* Show a simple example app connected to the REST API
* H2O, Keras, XGBoost, and DataRobot
* Add a DataRobot remote agent if you are interested in further model monitoring



In [ ]:
#Clone the repository
!git clone https://github.com/datarobot-community/mlops-examples

In [ ]:
!pip install -r /content/mlops-examples/'MLOps DRUM'/requirements.txt

## Train a regression model

A simple RandomForestRegressor to predict house prices in Boston.

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import pickle
import datetime

# Read the train and test data
TRAIN_DATA_REG = "/content/mlops-examples/MLOps DRUM/data/boston_housing_train.csv"  # 14 features
TEST_DATA_REG = "/content/mlops-examples/MLOps DRUM/data/boston_housing_test.csv"  # 13 features - target is removed

reg_X_train = pd.read_csv(TRAIN_DATA_REG)
reg_Y_train = reg_X_train.pop('MEDV')

reg_X_test = pd.read_csv(TEST_DATA_REG)

# Fit the model
rf = RandomForestRegressor()
rf.fit(reg_X_train, reg_Y_train)

# Pickle the file and write it to the file system
with open("/content/mlops-examples/MLOps DRUM/custom_model_reg/reg_rf_model.pkl", 'wb') as pkl:
    pickle.dump(rf, pkl)
    
# Call predict to confirm it works
rf.predict(reg_X_test)

array([25.941, 22.129, 34.559, 33.845, 35.273, 27.624, 21.774, 23.32 ,
       16.947])

## Testing the Model

Input the prediction dataset that includes all features except the target feature.

In [5]:
%%sh 
drum perf-test --code-dir /content/mlops-examples/'MLOps DRUM'/custom_model_reg --input /content/mlops-examples/'MLOps DRUM'/data/boston_housing_test.csv  --target-type regression

DRUM performance test
Model:      /Users/thodoris.petropoulos/github/mlops-examples-wip/MLOps DRUM/custom_model_reg
Data:       /Users/thodoris.petropoulos/github/mlops-examples-wip/MLOps DRUM/data/boston_housing_test.csv
# Features: 13
Preparing test data...



Running test case with timeout: 600
Running test case: 72 bytes - 1 samples, 100 iterations
Running test case with timeout: 600
Running test case: 0.1MB - 1449 samples, 50 iterations
Running test case with timeout: 600
Running test case: 10MB - 144964 samples, 5 iterations
Running test case with timeout: 600
Running test case: 50MB - 724823 samples, 1 iterations
Test is done stopping drum server

  size     samples   iters    min     avg     max    used (MB)   total physical 
                                                                      (MB)      
72 bytes         1     100   0.015   0.016   0.060     140.348         16384.000
0.1MB         1449      50   0.024   0.025   0.033     143.691         16384.000
10MB        1

tput: terminal attributes: Device not configured



## Validating the Model

In [6]:
%%sh 
drum validation --code-dir /content/mlops-examples/'MLOps DRUM'/custom_model_reg \
--input /content/mlops-examples/'MLOps DRUM'/data/boston_housing_test.csv \
--target-type regression > drum_validation.log

In [7]:
%%sh
tail drum_validation.log



Validation checks results
      Test case          Status   Details
Basic batch prediction   PASSED          
Null value imputation    PASSED          


# Batch Scoring with DRUM
<a id="setup_complete"></a>

At this point our model has been written to disk and we want to start making predictions with it.  To do this, we'll leverage DRUM and it's ability to natively handle our scikit learn model, all we need to do is tell DRUM where it resides as well as the data we wish to score.  

There are a lot of frameworks which DRUM supports nateively, but for those which DRUM doesn't support of these shelf, we'll just need to create some custom hooks so DRUM.  In this example, we'll highlight some very simple custom hooks, and will provide links to more complex examples.  

In [10]:
%%sh 
drum score --code-dir /content/mlops-examples/'MLOps DRUM'/custom_model_reg \
--input /content/mlops-examples/'MLOps DRUM'/data/boston_housing_test.csv \
--output /content/mlops-examples/'MLOps DRUM'/data/predictions.csv --target-type regression

In [11]:
pd.read_csv("/content/mlops-examples/'MLOps DRUM'/data/predictions.csv").head()

,Predictions
0,25.941
1,22.129
2,34.559
3,33.845
4,35.273


# Start the inference server locally

Batch scoring can be very useful, but the utility DRUM offers does not stop there.  We can also leverage DRUM to serve our model as a RESTful API endpoint.  The only thing that changes is the way we will structure the command - using the `server` mode instead of `score` model.  We'll also need to provide an address which is NOT in use.  

When starting the server, we'll use `subprocess.Popen` so we may interact with the server in this notebook

In [13]:
import subprocess
import requests
import pandas as pd
from io import BytesIO
import yaml
import time
import os
import datarobot as dr
from pprint import pprint

In [15]:
run_inference_server = ["drum",
              "server",
              "--code-dir","/content/mlops-examples/'MLOps DRUM'/custom_model_reg", 
              "--address", "0.0.0.0:6789", 
              "--show-perf",
              "--target-type", "regression",
              "--logging-level", "info",
              "--show-stacktrace",
              "--verbose",
              "--production", 
              "--max-workers", "5"
              ]

In [16]:
inference_server = subprocess.Popen(run_inference_server, stdout=subprocess.PIPE)

In [17]:
!sudo service nginx status

Password:
sudo: a password is required


In [18]:
## confirm the server is running
time.sleep(5) ## snoozing before pinging the server to give it time to actually start
print('check status')
requests.request("GET", "http://0.0.0.0:6789/").content

check status


ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=6789): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feed4ba61d0>: Failed to establish a new connection: [Errno 61] Connection refused'))

## Send data to server for inference

The request must provide our dataset as form data.  In order to do so, we'll create a simple python function to pass the data over appropriately.  We'll leverage the same function in our simple flask app a little later.  

In [ ]:
def score(data, port = "6789"):
    b_buf = BytesIO()
    b_buf.write(data.to_csv(index=False).encode("utf-8"))
    b_buf.seek(0)
  
    url = "http://localhost:{}/predict/".format(port)
    files = [
        ('X', b_buf)
    ]
    response = requests.request("POST", url, files = files, timeout=None, verify=False)
    return response

In [ ]:
# %%timeit
scoring_data = pd.read_csv("/content/mlops-examples/'MLOps DRUM'/data/boston_housing_test.csv")
predictions = score(scoring_data).json() ## score entire dataset but only show first 5 records
pprint(predictions)

In [ ]:
requests.request("GET", "http://0.0.0.0:6789/").content

In [ ]:
inference_server.terminate()
inference_server.stdout.readlines()

In [ ]:
%%sh
nginx -s stop
sudo service nginx status

## Value Prop

One may ask, what is the benefit to be had here?  Well, first of, there is not need for me to write an api to get the model up and running.  Second, DRUM allows me to abstract the framework away (provided I'm using one that is natively supported, or I can write enough python so that DRUM understands how to hook up to the model.  

For example, I could hot swap models as I see fit (see exampels in `./src/other_models`)

While we will run through several other frameworks with in `score` you can bet they are supported in `server` mode as well!

#### H2o Mojo

In [20]:
!drum score --code-dir /content/mlops-examples/'MLOps DRUM'/other_models/h2o_mojo/regression --input /content/mlops-examples/'MLOps DRUM'/data/boston_housing_test.csv --target-type regression

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
   Predictions
0    24.504000
1    22.492000
2    34.554001
3    34.420001
4    35.289001
5    28.394001
6    21.936000
7    23.451000
8    17.065000


#### Keras

In [21]:
!drum score --code-dir /content/mlops-examples/'MLOps DRUM'/other_models/python3_keras_joblib --input /content/mlops-examples/'MLOps DRUM'/data/boston_housing_test.csv --target-type regression



2021-05-27 16:49:14,280 ERROR drum.PythonModelAdapter:  Could not load hooks: No module named 'tensorflow'



Running environment language: Python.
Failed loading hooks from [/Users/thodoris.petropoulos/github/mlops-examples-wip/MLOps DRUM/other_models/python3_keras_joblib/custom.py] : No module named 'tensorflow'


#### XGBoost

In [ ]:
!drum score --code-dir /content/mlops-examples/'MLOps DRUM'/other_models/python3_xgboost --input /content/mlops-examples/'MLOps DRUM'/data/boston_housing_test.csv --target-type regression


#### DataRobot Codegen

In [ ]:
!drum score --code-dir /content/mlops-examples/'MLOps DRUM'/other_models/dr_codegen --input /content/mlops-examples/'MLOps DRUM'/data/boston_housing_test.csv --target-type regression
